<a href="https://colab.research.google.com/github/jasonmcaffee/llm-playground/blob/main/validate_jasonmcaffee_flan_t5_large_samsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install accelerate
!pip install loralib
!pip install evaluate
!pip install deepspeed
!pip install tqdm
!pip install datasets
!pip install peft
!pip install py7zr #to use samsum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 3.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [ ]:
# Load the jasonmcaffee/flan-t5-large-samsum model and tokenizer
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
peft_model_id = "jasonmcaffee/flan-t5-large-samsum"
config = PeftConfig.from_pretrained(peft_model_id)

#https://huggingface.co/docs/transformers/main/en/main_classes/quantization
#load_in_8bit=True You can load a model by roughly halving the memory requirements by using load_in_8bit=True argument when calling .from_pretrained method
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, load_in_8bit=True,  device_map="auto") 
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map="auto")
model.eval()

print("Peft model loaded")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Peft model loaded


In [ ]:
dialogue = """A taconachomonstera rides upon the greatest purple mushroom the world has ever laid eyese upon.   The task of cheesing the burrito is a common ritual amongst the tortilla chips.   A brontasaurustops resides over the nightly festivities, diving into a pool of sour cream while singing "children of the enchilladas, I emplore thee to hearken unto my words and partake in cheesing the burrito" 
"""
# dialogue = """The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.
# """
# From another samsum model: https://huggingface.co/stacked-summaries/flan-t5-large-stacked-samsum-1024?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+was+the+first+structure+to+reach+a+height+of+300+metres.+Due+to+the+addition+of+a+broadcasting+aerial+at+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct.
# The Eiffel Tower is the second tallest free-standing structure in France after the Millau

input_ids = tokenizer(dialogue, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(
    input_ids=input_ids, 
    min_length=20,
    max_new_tokens=100, 
    length_penalty=1.9, #Exponential penalty to the length that is used with beam-based generation. It is applied as an exponent to the sequence length, which in turn is used to divide the score of the sequence. Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    num_beams=4,
    temperature=0.9,
    top_k=150, # default 50
    repetition_penalty=2.1,
    # do_sample=True, 
    top_p=0.9,
    )
print(f"input sentence: {dialogue}\n{'---'* 20}")

summarization = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

print(f"summary:\n{summarization}")
#The Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.


input sentence: A taconachomonstera rides upon the greatest purple mushroom the world has ever laid eyese upon.   The task of cheesing the burrito is a common ritual amongst the tortilla chips.   A brontasaurustops resides over the nightly festivities, diving into a pool of sour cream while singing "children of the enchilladas, I emplore thee to hearken unto my words and partake in cheesing the burrito" 

------------------------------------------------------------
summary:
A taconachomonstera rides upon the greatest purple mushroom the world has ever laid eyes upon. The task of cheesing the burrito is a common ritual


In [ ]:
model_with_no_samsum = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large", load_in_8bit=True,  device_map="auto") 
tokenizer_with_no_samsum = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [ ]:
input_ids_with_no_samsum = tokenizer_with_no_samsum(dialogue, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs_with_no_samsum = model_with_no_samsum.generate(
    input_ids=input_ids_with_no_samsum, 
    min_length=20,
    max_new_tokens=100, 
    length_penalty=1.9, #Exponential penalty to the length that is used with beam-based generation. It is applied as an exponent to the sequence length, which in turn is used to divide the score of the sequence. Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    num_beams=4,
    temperature=0.9,
    top_k=150, # default 50
    repetition_penalty=2.1,
    # do_sample=True, 
    top_p=0.9,
    )
print(f"input sentence: {dialogue}\n{'---'* 20}")

summarization_with_no_samsum = tokenizer_with_no_samsum.batch_decode(outputs_with_no_samsum.detach().cpu().numpy(), skip_special_tokens=True)[0]

print(f"summary_with_no_samsum:\n{summarization_with_no_samsum}")
#The Eiffel Tower is the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930.

input sentence: A taconachomonstera rides upon the greatest purple mushroom the world has ever laid eyese upon.   The task of cheesing the burrito is a common ritual amongst the tortilla chips.   A brontasaurustops resides over the nightly festivities, diving into a pool of sour cream while singing "children of the enchilladas, I emplore thee to hearken unto my words and partake in cheesing the burrito" 

------------------------------------------------------------
summary_with_no_samsum:
a taconachomonstera rides upon the greatest purple mushroom the world has ever laid eyese upon. The task of cheesing the burrito is a common ritual amongst the tortilla chips. A brontasaurustops resides over the nightly festivities, diving into a pool of sour cream while singing "children of the enchilladas, I emplore thee to hearken unto my
